In [1]:
from typing import List, Dict, Optional
import urllib.parse
import json
import httpx
import logging


logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

## Base URL for the API
For API documentation, see https://c-stac-api.c-core.app/docs

In [2]:
api_url = "https://c-stac-api.c-core.app"

## List all collection IDs in the catalog

In [3]:
def list_collection_ids(url: str = api_url) -> List[str]:
    """List collection IDs in the catalog."""
    get_url = f"{url}/collections"
    
    with httpx.Client(timeout = 20) as client:
        response = client.get(get_url)
    
    # Raise exception if status code not 2xx success
    response.raise_for_status()
    
    response_body = response.json()
    collections = response_body["collections"]
    collection_ids = [collection["id"] for collection in collections]
    
    return collection_ids
    

In [4]:
list_collection_ids()

['joplin',
 'ice-drift-feature-tracking',
 'floe-edge-convergence',
 'floe-edge-coherence',
 'floe-edge-linestrings',
 'floe-edge-polygons']

## List items in collection

In [5]:
def list_items( 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[str] = "-id",
    url: str = api_url) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""
    # `sortby` need to be urlencoded because it accepts `+`
    url_parameters = f"collections={collection_id}&limit={limit}&sortby={urllib.parse.quote(sortby)}"
    get_collections_url = f"{url}/search?{url_parameters}"
    
    with httpx.Client(timeout = 20) as client:
        log.info(get_collections_url)
        response = client.get(get_collections_url)
    
    # Raise exception if status code not 2xx success
    response.raise_for_status()
    
    response_body = response.json()
    
    return response_body

In [6]:
items = list_items("floe-edge-linestrings", sortby="-id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&limit=2&sortby=-id


['2021-12-28T12:34:53+00:00', '2021-12-24T13:07:39+00:00']

In [7]:
items = list_items("floe-edge-linestrings", sortby="+id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&limit=2&sortby=%2Bid


['2019-12-16T12:25:42+00:00', '2019-12-18T12:25:42+00:00']

## Filter by location with bbox
For further API details, see https://api.stacspec.org/v1.0.0-rc.1/item-search/#operation/getItemSearch

In [8]:
def list_items_in_bbox(
    bbox: List[float], 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[str] = "-id",
    url: str = api_url) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""
    bbox_parameter = ",".join([str(coordinate) for coordinate in bbox])
    
    # `sortby` need to be urlencoded because it accepts `+`
    url_parameters = f"collections={collection_id}&bbox={bbox_parameter}&limit={limit}&sortby={urllib.parse.quote(sortby)}"
    get_collections_url = f"{url}/search?{url_parameters}"
    
    with httpx.Client(timeout = 20) as client:
        log.info(get_collections_url)
        response = client.get(get_collections_url)
    
    # Raise exception if status code not 2xx success
    response.raise_for_status()
    
    response_body = response.json()
    
    return response_body

In [9]:
bboxes = {
    "resolute": [-97.1, 74.3, -92.5, 75.0],
    "pond_inlet": [-80.6, 72.2, -75.1, 73.2],
    "world": [-180, -90, 180, 90],
}

In [10]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="-id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=-id


['2021-12-24T11:30:21+00:00', '2021-12-19T12:10:31+00:00']

In [11]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="+id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=%2Bid


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

In [12]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="-datetime")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=-datetime


['2021-12-24T11:30:21+00:00', '2021-12-19T12:10:31+00:00']

In [13]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="+datetime")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=%2Bdatetime


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

## Filter by location with geojson
For further API details, see https://api.stacspec.org/v1.0.0-rc.1/item-search/#operation/postItemSearch

In [14]:
geojsons = {
    "tutoyaktuk": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -97.09716796875,
              74.27612190544454
            ],
            [
              -92.5103759765625,
              74.27612190544454
            ],
            [
              -92.5103759765625,
              75.00636121985819
            ],
            [
              -97.09716796875,
              75.00636121985819
            ],
            [
              -97.09716796875,
              74.27612190544454
            ]
          ]
        ]
    },
    "pond_inlet": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -80.606689453125,
              72.18852591070342
            ],
            [
              -75.1025390625,
              72.18852591070342
            ],
            [
              -75.1025390625,
              73.18543401519665
            ],
            [
              -80.606689453125,
              73.18543401519665
            ],
            [
              -80.606689453125,
              72.18852591070342
            ]
          ]
        ]
    },
    "world": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -180,
              -90
            ],
            [
              180,
              -90
            ],
            [
              180,
              90
            ],
            [
              -180,
              90
            ],
            [
              -180,
              -90
            ]
          ]
        ]
    }
}

In [26]:
def list_items_in_geojson(
    geojson: Dict, 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[str] = "-id",
    url: str = api_url) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""

    post_url = f"{url}/search"
    
    data = {
        "intersects": geojson,
        "collections": [collection_id],
        "limit": limit,
        "sortby": [sortby],
    }
    
    with httpx.Client(timeout = 20) as client:
        log.info(post_url)
        log.info(data)
        response = client.post(post_url, data=json.dumps(data))
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

In [27]:
items = list_items_in_geojson(geojsons["pond_inlet"], "floe-edge-linestrings", sortby="-datetime")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-80.606689453125, 72.18852591070342], [-75.1025390625, 72.18852591070342], [-75.1025390625, 73.18543401519665], [-80.606689453125, 73.18543401519665], [-80.606689453125, 72.18852591070342]]]}, 'collections': ['floe-edge-linestrings'], 'limit': 2, 'sortby': ['-datetime']}
ERROR:__main__:{'detail': [{'loc': ['body', 'sortby', 0], 'msg': 'value is not a valid dict', 'type': 'type_error.dict'}]}
Traceback (most recent call last):
  File "<ipython-input-26-9c792de08ff6>", line 25, in list_items_in_geojson
    response.raise_for_status()
  File "/home/jthetzel/.cache/pypoetry/virtualenvs/notebooks-4iuYC0qS-py3.9/lib/python3.9/site-packages/httpx/_models.py", line 1510, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '400 Bad Request' for url 'https://c-stac-api.c-core.app/search'
For more information c

HTTPStatusError: Client error '400 Bad Request' for url 'https://c-stac-api.c-core.app/search'
For more information check: https://httpstatuses.com/400